In [1]:
## SETUP:
from trans_tools import *
from IrisBackendv3.codec.payload import CommandPayload
from IrisBackendv3.codec.packet import IrisCommonPacket
from IrisBackendv3.codec.metadata import DataPathway, DataSource
from IrisBackendv3.codec.magic import Magic
from IrisBackendv3.utils.basic import bytearray_to_spaced_hex as hexstr

wired_seq_num = 0x01
wireless_seq_num = 0x01
# steps = ['heater-test','setup', 'power-on', 'wifi-mode', 'deploy']
itr = 0

In [2]:
## SETTINGS:

serial_device = '/dev/ttyUSB0'
ip="192.168.1.2"
port=8080

In [3]:
settings['SAVE_FILE_PREFIX'] = 'iris_firing_test_trial0' # this the prefix on all log files. make it something unique.
load_cache()

In [4]:
## SET TO TRUE TO VIEW A FULL-LIST OF ALL COMMANDS AND TELEMETRY:
if show_commands := False:
    standards.print_overview()

## **Testing Procedure:** picking pre-prepared commands

**Running the Code:**

- On the first run **OR** after resetting / power-cycling the board, **pick a step in the next cell** then press the `red square`, then `green circle`, then `double arrows` to run all the code.
- After changing to the next step (in the next cell), **click here** (yes, this text cell that you're currently reading) and then **press only** the `red square` then the `play button with the down arrow` to send the new command without resetting the sequence numbers.
- You can read wired telemetry streams at the bottom of this notebook and check for the presence of wireless telemetry in wireshark using the filter: `udp && udp.port==8080 && not icmp` . 

**Steps:**

0. Run `setup`, wait for watchdog heartbeats to verify that the mode is now `RS_SERVICE`.
1. Run `power-on`, verify Hercules has been turned on by the presense of `ICP` telemetry packets in the wired output.
2. Run `wifi-mode` to tell Hercules to downlink telemetry wirelessly instead of over the wire. Verify that `ICP` telemetry packets are now showing up in the wireless output. Note: there should no longer be a wired output showing. Also note: wired and wireless commands will still work until the wired connection has been physically disconnected.
3. Run `deploy` to tell Hercules to tell the Watchdog to release its deployment interlock and then have Hercules release its own interlock. Verify that both interlocks are released and that the deployment switch is now closed by removing the deployment power leads from the deployment power supply and verifying that the resistance between them is on the order of single digit Ohms instead of Megaohms.

Once `deploy` has been successfully issued, applying deployment power to the rover will cause its deployment. **Note:** to properly simulate mission conditions, the interlocks should be released and deployment switch closed **before** applying deployment power.

**Troubleshooting:**

**Note:** If you have issues running the `deploy` command, you can instead run the `deploy-wd-only` and the `deploy-herc-only` commands to directly tell the Watchdog and Hercules to release their interlocks separately.

**Note:** If you're having trouble getting wireless comms working, make sure this computer's IP matches the `spacecraft_ip` used by Hercules (usually `192.168.1.120`) and that the IP you're sending to (configured at the top) matches the `rover_ip` used by the Hercules.

In [5]:
## Pick the step you want to run:

steps = ['heater-test', 'setup', 'power-on', 'wifi-mode', 'deploy']
step = steps[itr]

In [6]:
## Pre-prepared commands list:

source = DataSource.GENERATED

prepared_commands = {
    'setup': ( # Tell the Watchdog to switch into service mode
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_SwitchToServiceMode',
        dict(confirm='CONFIRM_SERVICE'),
        DataPathway.WIRED
    ),
    'power-on': ( # Turn Everything On
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_PrepareForDeployment',
        dict(confirm='CONFIRM_PREP'),
        DataPathway.WIRED
    ),
    'wifi-mode': ( # Turn Everything On
        DataPathway.WIRED,
        Magic.COMMAND, # "normal" command is for Hercules
        'GroundInterface_SetPrimaryInterface',
        dict(primary_interface='WF_121'),
        DataPathway.WIRELESS
    ),
    'deploy': (
        DataPathway.WIRED,
        Magic.COMMAND, # "normal" command is for Hercules
        'WatchDogInterface_DisengageFromLander',
        dict(confirm='CONFIRM_DEPLOY'),
        DataPathway.WIRED
    ),
    'deploy-wd-only': ( # special command to tell only WD to release its interlock (in case Herc-WD comms are broken)
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_DisengageFromLander',
        dict(confirm='CONFIRM_DEPLOY'),
        DataPathway.WIRELESS
    ),
    'deploy-herc-only': ( # special command to tell only Herc to release its interlock (in case Herc-WD comms are broken)
        DataPathway.WIRELESS,
        Magic.COMMAND, # "normal" command is for Hercules
        'WatchDogInterface_DisengageFromLander',
        dict(confirm='CONFIRM_DEPLOY'),
        DataPathway.WIRELESS
    ),
    'heater-test': (
        DataPathway.WIRED,
        Magic.COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='ENABLE_HEATER_CONTROL'),
        DataPathway.WIRELESS
    )
}
pathway, magic, command_name, kwargs, telem_pathway = prepared_commands[step]

In [7]:
## Build Command:

if pathway == DataPathway.WIRED:
    wired_seq_num += 1
    seq_num = wired_seq_num
elif pathway == DataPathway.WIRELESS:
    wireless_seq_num += 1
    seq_num = wireless_seq_num

command_payload_type = {
    Magic.WATCHDOG_COMMAND: WatchdogCommandPayload,
    Magic.COMMAND: CommandPayload
}[magic]

module, command = standards.global_command_lookup(command_name)
payloads = PayloadCollection(
    CommandPayload=[
        command_payload_type(
            pathway=pathway,
            source=source,
            magic=magic,
            module_id=module.ID,
            command_id=command.ID,
            args=kwargs
        )
    ],
    TelemetryPayload=[],
    EventPayload=[],
    FileBlockPayload=[]
)
packet = IrisCommonPacket(
    seq_num = seq_num,
    payloads = payloads,
    pathway = pathway,
    source = source
).encode()
print(hexstr(packet))

02 0a 00 db 55 da ba 00 00 10 1f 00 00 00


In [8]:
## Send Command:
connect_serial(device = serial_device)
if pathway == DataPathway.WIRED:
    send_data_wd_serial(packet)
elif pathway == DataPathway.WIRELESS:
    send_wifi(packet, ip=ip, port=port)

Connection Success!


In [9]:
## Stream Telemetry:
if True or telem_pathway == DataPathway.WIRED:
    stream_data_ip_udp_serial()
elif telem_pathway == DataPathway.WIRELESS:
    pass # Check for wireless telemetry in Wireshark. For parsing help, run: `pyenv exec python parse_pcap.py --help`


RS_KEEPALIVE:	[Heat: ON, Ctrl: ON] 	260.9°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 8500/65535
RS_KEEPALIVE:	[Heat: ON, Ctrl: ON] 	260.9°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 8500/65535
RS_KEEPALIVE:	[Heat: ON, Ctrl: ON] 	260.9°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 8500/65535
RS_KEEPALIVE:	[Heat: ON, Ctrl: ON] 	260.9°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 8500/65535
An otherwise unresolved error occurred during packet streaming: unsupported operand type(s) for -: 'NoneType' and 'int'
An otherwise unresolved error occurred during packet streaming: cannot convert 'NoneType' object to bytes
